In [1]:
import sys
sys.path.append("../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from models import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
def evaluateCorInfoMaxV3(model, loader, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                         neural_lr_decay_multiplier,
                         neural_dynamic_iterations, device, printing = True):
    # Evaluate the model on a dataloader with T steps for the dynamics
    #model.eval()
    correct=0
    phase = 'Train' if loader.dataset.train else 'Test'
    
    for x, y in loader:
        x = x.view(x.size(0),-1).to(device).T
        y = y.to(device)
        
        neurons = model.fast_forward(x)
        
        # dynamics for T time steps
        neurons = model.run_neural_dynamics(x, y_one_hot, neurons, neural_lr_start, neural_lr_stop, 
                                            neural_lr_rule,
                                            neural_lr_decay_multiplier, neural_dynamic_iterations, 0, "test")
        pred = torch.argmax(neurons[-1], dim=0).squeeze()  # in this case prediction is done directly on the last (output) layer of neurons
        correct += (y == pred).sum().item()

    acc = correct/len(loader.dataset) 
    if printing:
        print(phase+' accuracy :\t', acc)   
    return acc

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.0,), std=(1.0,))])

mnist_dset_train = torchvision.datasets.MNIST('./data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('./data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [5]:
activation = hard_sigmoid
architecture = [784, 500, 10]

x,y = next(iter(train_loader))
x = x.view(x.size(0),-1).to(device).T
y_one_hot = F.one_hot(y, 10).to(device).T

beta = 1
lambda_ = 0.9999
epsilon = 0.5#0.5
one_over_epsilon = 1 / epsilon
lr_start = {'ff' : 0.01, 'fb': 0.001, 'lat': 1e-3}
neural_lr_start = 0.25
neural_lr_stop = 1e-3
neural_lr_rule = "divide_by_slow_loop_index"
neural_lr_decay_multiplier = 0.1
neural_dynamic_iterations = 50
output_sparsity = True
STlambda_lr = 0.005
model = CorInfoMax( architecture = architecture, lambda_ = lambda_,
                    epsilon = epsilon, activation = activation,
                    output_sparsity = output_sparsity, STlambda_lr = STlambda_lr
                  )

In [6]:
evaluateCorInfoMaxV3(model, test_loader, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                         neural_lr_decay_multiplier,
                         neural_dynamic_iterations, device, printing = True)

Test accuracy :	 0.1065


0.1065

In [ ]:
trn_acc_list = []
tst_acc_list = []

n_epochs = 50

for epoch_ in range(n_epochs):
    lr = {'ff' : lr_start['ff'] * (0.99)**epoch_, 'fb' : lr_start['fb'] * (0.99)**epoch_}
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        x = x.view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T

        _ = model.batch_step(  x, y_one_hot, lr, neural_lr_start, neural_lr_stop, neural_lr_rule,
                                    neural_lr_decay_multiplier, neural_dynamic_iterations, beta)

    trn_acc = evaluateCorInfoMaxV3(  model, train_loader, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                                     neural_lr_decay_multiplier,
                                     neural_dynamic_iterations, device, printing = False)
    tst_acc = evaluateCorInfoMaxV3(  model, test_loader, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                                     neural_lr_decay_multiplier,
                                     neural_dynamic_iterations, device, printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

3000it [00:54, 54.57it/s]
5it [00:00, 48.66it/s]

Epoch : 1, Train Accuracy : 0.7520666666666667, Test Accuracy : 0.7564


3000it [00:55, 54.48it/s]
5it [00:00, 48.97it/s]

Epoch : 2, Train Accuracy : 0.8070833333333334, Test Accuracy : 0.8156


3000it [00:54, 54.62it/s]
5it [00:00, 49.65it/s]

...

3000it [00:54, 54.88it/s]
5it [00:00, 49.65it/s]

Epoch : 18, Train Accuracy : 0.8408166666666667, Test Accuracy : 0.8451


3000it [00:54, 54.75it/s]
5it [00:00, 49.19it/s]

Epoch : 19, Train Accuracy : 0.8411833333333333, Test Accuracy : 0.8453


3000it [00:54, 54.67it/s]
